Copia arhcivos InBOUND SIEMPRE Y CUANDO NO HAYA OTRO CON LA MISMA FECHA EN LOS PRIMEROS 7 CARACTERES DEL NOMBRE

In [1]:
import os
import shutil

ruta_origen = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentos\Planificación y Compras AFM\Forecast Inbound"
ruta_destino = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentoss\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Historico Inbound"

for root, dirs, files in os.walk(ruta_origen):
    for file in files:
        if "Inbound" in file:
            ruta_archivo = os.path.join(root, file)
            nombre_destino = os.path.join(ruta_destino, file)
            existe_similar = any(f.startswith(file[:7]) for f in os.listdir(ruta_destino))
            if not os.path.exists(nombre_destino) and not existe_similar:
                shutil.copy(ruta_archivo, ruta_destino)


Transformacion de los inbound

In [2]:
import os
import pandas as pd

ruta_excel = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentoss\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Historico Inbound"
ruta_parquet = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentoss\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Historico Inbound\Parquet_Inbound_Histórico"

os.makedirs(ruta_parquet, exist_ok=True)

errores = []

for archivo in os.listdir(ruta_excel):
    if archivo.endswith(".xlsx"):
        nombre_archivo = archivo[:60].replace(' ', '_')
        nombre_corto = archivo[:100]
        archivo_parquet = f"{nombre_archivo}.parquet"
        ruta_archivo_parquet = os.path.join(ruta_parquet, archivo_parquet)

        try:
            if not os.path.exists(ruta_archivo_parquet):
                ruta_archivo_excel = os.path.join(ruta_excel, archivo)
                hojas = pd.ExcelFile(ruta_archivo_excel).sheet_names
                hoja_correcta = next((hoja for hoja in hojas if hoja.lower() == 'base'), None)

                if hoja_correcta:
                    df = pd.read_excel(ruta_archivo_excel, sheet_name=hoja_correcta, skiprows=2)
                    df = df.iloc[:, :206]
                    df = df.astype(str)
                    df["Fecha_FC"] = nombre_corto
                    df.to_parquet(ruta_archivo_parquet, index=False)
                else:
                    errores.append(f"No se encontró la hoja 'base' en {archivo}")
        except Exception as e:
            errores.append(f"Error al procesar {archivo}: {str(e)}")

if errores:
    print("Archivos con problemas:")
    for error in errores:
        print(error)
else:
    print("Todos los archivos fueron procesados correctamente.")


Todos los archivos fueron procesados correctamente.


In [3]:
import polars as pl
import os

ruta = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentoss\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Historico Inbound\Parquet_Inbound_Histórico"

archivos = [f for f in os.listdir(ruta) if f.endswith('.parquet')]

for i, archivo in enumerate(archivos, start=1):
    globals()[f"df{i}"] = pl.read_parquet(os.path.join(ruta, archivo))
    
# BORRAR LA COLUMNA 2

for i in range(1, len(archivos) + 1):
    df = globals()[f"df{i}"]
    globals()[f"df{i}"] = df.drop(df.columns[2])


In [4]:
# print(df6.columns[167:171])
# print(df7.columns[167:171])


['2025-05-01 00:00:00.6', '2025-06-01 00:00:00.5', '2024-07-01 00:00:00.8', '2024-08-01 00:00:00.7']
['2025-05-01 00:00:00.6', '2025-06-01 00:00:00.6', '2024-08-01 00:00:00.7', '2024-09-01 00:00:00.7']


LEER COLS

In [4]:
for i, archivo in enumerate(archivos, start=1):
    df = globals()[f"df{i}"]
    print(f"DataFrame df{i} - Archivo: {archivo[:10]} - Columnas desde la 169 en adelante:")
    print(df.columns[168:])
    print("-" * 50)

DataFrame df1 - Archivo: 01.2024_S& - Columnas desde la 169 en adelante:
['2025-01-01 00:00:00.5', '2024-02-01 00:00:00.8', '2024-03-01 00:00:00.8', '2024-04-01 00:00:00.8', '2024-05-01 00:00:00.8', '2024-06-01 00:00:00.8', '2024-07-01 00:00:00.8', '2024-08-01 00:00:00.7', '2024-09-01 00:00:00.7', '2024-10-01 00:00:00.7', '2024-11-01 00:00:00.7', '2024-12-01 00:00:00.7', '2025-01-01 00:00:00.6', '2024-02-01 00:00:00.9', '2024-03-01 00:00:00.9', '2024-04-01 00:00:00.9', '2024-05-01 00:00:00.9', '2024-06-01 00:00:00.9', '2024-07-01 00:00:00.9', '2024-08-01 00:00:00.8', '2024-09-01 00:00:00.8', '2024-10-01 00:00:00.8', '2024-11-01 00:00:00.8', '2024-12-01 00:00:00.8', '2025-01-01 00:00:00.7', '2023-09-01 00:00:00.3', '2023-10-01 00:00:00.3', '2023-11-01 00:00:00.3', '2023-12-01 00:00:00.3', '2024-01-01 00:00:00.2', '2024-02-01 00:00:00.10', '2024-03-01 00:00:00.10', '2024-04-01 00:00:00.10', '2024-05-01 00:00:00.10', '2024-06-01 00:00:00.10', '2024-07-01 00:00:00.10', '2024-08-01 00:00:00

LEER PARA CONCATENAR FC FINAL, VISUALIZAR COLUMNAS LUEGO COCNATENAR

In [5]:
for i, archivo in enumerate(archivos, start=1):
    df = globals()[f"df{i}"]
    print(f"DataFrame df{i} - Archivo: {archivo[:10]} - Columnas desde la 169 en adelante:")
    print(df.columns[1:100])
    print("-" * 50)


DataFrame df1 - Archivo: 01.2024_S& - Columnas desde la 169 en adelante:
['Ult. Eslabón', 'Agrupación', 'Sub Agrupación', 'Categoría', 'Sub Categoría', 'Descripcion Material', 'Marca', 'Sector', 'Vig', 'Nuevo', 'Season', 'Valor Libro', 'Canal', '2022-07-01 00:00:00', '2022-08-01 00:00:00', '2022-09-01 00:00:00', '2022-10-01 00:00:00', '2022-11-01 00:00:00', '2022-12-01 00:00:00', '2023-01-01 00:00:00', '2023-02-01 00:00:00', '2023-03-01 00:00:00', '2023-04-01 00:00:00', '2023-05-01 00:00:00', '2023-06-01 00:00:00', '2023-07-01 00:00:00', '2023-08-01 00:00:00', '2023-09-01 00:00:00', '2023-10-01 00:00:00', '2023-11-01 00:00:00', '2023-12-01 00:00:00', '2022-01-01 00:00:00', '2022-02-01 00:00:00', '2022-03-01 00:00:00', '2022-04-01 00:00:00', '2022-05-01 00:00:00', '2022-06-01 00:00:00', '2022-07-01 00:00:00.1', '2022-08-01 00:00:00.1', '2022-09-01 00:00:00.1', '2022-10-01 00:00:00.1', '2022-11-01 00:00:00.1', '2022-12-01 00:00:00.1', '2023-01-01 00:00:00.1', '2023-02-01 00:00:00.1', '20

CREAR DF'S DE FC FINAL

In [6]:
for i, archivo in enumerate(archivos, start=1):
    df = globals()[f"df{i}"]
    print(f"DataFrame df{i} - Archivo: {archivo[:10]} - Columnas desde la 169 en adelante:")
    print(df.columns[1:100])
    print("-" * 50)
    
    # Validar que el DataFrame tiene al menos 169 columnas
    if len(df.columns) >= 169:
        # Seleccionar las primeras 13 columnas, las columnas desde la 169 hasta 169+12, y la última columna
        columnas_seleccionadas = list(df.columns[:13]) + list(df.columns[169:169+12]) + [df.columns[-1]]
        globals()[f"df_fi_{i}"] = df.select(columnas_seleccionadas)
    else:
        print(f"DataFrame df{i} no tiene suficientes columnas para la operación.")


DataFrame df1 - Archivo: 01.2024_S& - Columnas desde la 169 en adelante:
['Ult. Eslabón', 'Agrupación', 'Sub Agrupación', 'Categoría', 'Sub Categoría', 'Descripcion Material', 'Marca', 'Sector', 'Vig', 'Nuevo', 'Season', 'Valor Libro', 'Canal', '2022-07-01 00:00:00', '2022-08-01 00:00:00', '2022-09-01 00:00:00', '2022-10-01 00:00:00', '2022-11-01 00:00:00', '2022-12-01 00:00:00', '2023-01-01 00:00:00', '2023-02-01 00:00:00', '2023-03-01 00:00:00', '2023-04-01 00:00:00', '2023-05-01 00:00:00', '2023-06-01 00:00:00', '2023-07-01 00:00:00', '2023-08-01 00:00:00', '2023-09-01 00:00:00', '2023-10-01 00:00:00', '2023-11-01 00:00:00', '2023-12-01 00:00:00', '2022-01-01 00:00:00', '2022-02-01 00:00:00', '2022-03-01 00:00:00', '2022-04-01 00:00:00', '2022-05-01 00:00:00', '2022-06-01 00:00:00', '2022-07-01 00:00:00.1', '2022-08-01 00:00:00.1', '2022-09-01 00:00:00.1', '2022-10-01 00:00:00.1', '2022-11-01 00:00:00.1', '2022-12-01 00:00:00.1', '2023-01-01 00:00:00.1', '2023-02-01 00:00:00.1', '20

In [7]:
for i in range(1, len(archivos) + 1):
    df_fi = globals()[f"df_fi_{i}"]
    print(f"DataFrame df_fi_{i} - Columnas:")
    print(df_fi.columns)
    print("-" * 50)


DataFrame df_fi_1 - Columnas:
['ID', 'Ult. Eslabón', 'Agrupación', 'Sub Agrupación', 'Categoría', 'Sub Categoría', 'Descripcion Material', 'Marca', 'Sector', 'Vig', 'Nuevo', 'Season', 'Valor Libro', '2024-02-01 00:00:00.8', '2024-03-01 00:00:00.8', '2024-04-01 00:00:00.8', '2024-05-01 00:00:00.8', '2024-06-01 00:00:00.8', '2024-07-01 00:00:00.8', '2024-08-01 00:00:00.7', '2024-09-01 00:00:00.7', '2024-10-01 00:00:00.7', '2024-11-01 00:00:00.7', '2024-12-01 00:00:00.7', '2025-01-01 00:00:00.6', 'Fecha_FC']
--------------------------------------------------
DataFrame df_fi_2 - Columnas:
['ID', 'Ult. Eslabón', 'Agrupación', 'Sub Agrupación', 'Categoría', 'Sub Categoría', 'Descripcion Material', 'Marca', 'Sector', 'Vig', 'Nuevo', 'Season', 'Valor Libro', '2024-02-01 00:00:00.8', '2024-03-01 00:00:00.8', '2024-04-01 00:00:00.8', '2024-05-01 00:00:00.8', '2024-06-01 00:00:00.8', '2024-07-01 00:00:00.8', '2024-08-01 00:00:00.7', '2024-09-01 00:00:00.7', '2024-10-01 00:00:00.7', '2024-11-01 00

# Melting del fc final

In [8]:
for i in range(1, len(archivos) + 1):
    df_fi = globals()[f"df_fi_{i}"]
    
    # Seleccionar las primeras 13 columnas y la última columna como índice
    index_vars = list(df_fi.columns[:13]) + [df_fi.columns[-1]]
    value_vars = [col for col in df_fi.columns if col not in index_vars]
    
    # Realizar unpivot para obtener el formato largo
    melted_df = df_fi.unpivot(index=index_vars, on=value_vars).rename({
        "variable": "Variable",
        "value": "Valor"
    })
    
    # Agregar columna "FC Type" con valor "FC Final"
    melted_df = melted_df.with_columns(pl.lit("FC Final").alias("FC Type"))
    
    # Guardar el resultado en una nueva variable dinámica
    globals()[f"melted_df_fi_{i}"] = melted_df
    
    # Imprimir el resultado
    print(f"Melt realizado para df_fi_{i} con columna 'FC Type':")
    print(melted_df)
    print("-" * 50)


Melt realizado para df_fi_1 con columna 'FC Type':
shape: (461_124, 17)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ ID        ┆ Ult.      ┆ Agrupació ┆ Sub Agrup ┆ … ┆ Fecha_FC  ┆ Variable  ┆ Valor     ┆ FC Type  │
│ ---       ┆ Eslabón   ┆ n         ┆ ación     ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ str       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ str       ┆ str       ┆ str       ┆ str      │
│           ┆ str       ┆ str       ┆ str       ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 1000144CL ┆ 1000144   ┆ ACCESORIO ┆ LIMPIEZA  ┆ … ┆ 01.2024   ┆ 2024-02-0 ┆ 0.0       ┆ FC Final │
│ MAYORISTA ┆           ┆ S         ┆ Y CUIDADO ┆   ┆ S&OP      ┆ 1 00:00:0 ┆           ┆          │
│           ┆           ┆           ┆           ┆   ┆ Demanda   ┆ 0.8       ┆           ┆          │
│           ┆      

LEER PARA CONCATENAR FC AJUSTADO

In [9]:
for i, archivo in enumerate(archivos, start=1):
    df = globals()[f"df{i}"]    
    if len(df.columns) >= 90:
        columnas_seleccionadas = list(df.columns[:13]) + list(df.columns[75:87]) + [df.columns[-1]]
        globals()[f"df_adj_{i}"] = df.select(columnas_seleccionadas)
    else:
        print(f"DataFrame df{i} no tiene suficientes columnas para la operación.")

for i in range(1, len(archivos) + 1):
    df_adj = globals()[f"df_adj_{i}"]
    
    # Seleccionar las primeras 13 columnas y la última columna como índice
    index_vars = list(df_adj.columns[:13]) + [df_adj.columns[-1]]
    value_vars = [col for col in df_adj.columns if col not in index_vars]
    
    # Realizar unpivot para obtener el formato largo
    melted_df = df_adj.unpivot(index=index_vars, on=value_vars).rename({
        "variable": "Variable",
        "value": "Valor"
    })
    
    # Agregar columna "FC Type" con valor "FC Final"
    melted_df = melted_df.with_columns(pl.lit("FC Ajustado").alias("FC Type"))
    
    # Guardar el resultado en una nueva variable dinámica
    globals()[f"melted_df_adj_{i}"] = melted_df
    
    # Imprimir el resultado
    print(f"Melt realizado para df_adj_{i} con columna 'FC Type':")
    print(melted_df)
    print("-" * 50)



Melt realizado para df_adj_1 con columna 'FC Type':
shape: (461_124, 17)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ ID        ┆ Ult.      ┆ Agrupació ┆ Sub Agrup ┆ … ┆ Fecha_FC  ┆ Variable  ┆ Valor     ┆ FC Type  │
│ ---       ┆ Eslabón   ┆ n         ┆ ación     ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ str       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ str       ┆ str       ┆ str       ┆ str      │
│           ┆ str       ┆ str       ┆ str       ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 1000144CL ┆ 1000144   ┆ ACCESORIO ┆ LIMPIEZA  ┆ … ┆ 01.2024   ┆ 2024-02-0 ┆ 0.0       ┆ FC       │
│ MAYORISTA ┆           ┆ S         ┆ Y CUIDADO ┆   ┆ S&OP      ┆ 1 00:00:0 ┆           ┆ Ajustado │
│           ┆           ┆           ┆           ┆   ┆ Demanda   ┆ 0.1       ┆           ┆          │
│           ┆     

LEER PARA CONCATENAR FC ACCION

In [10]:
for i, archivo in enumerate(archivos, start=1):
    df = globals()[f"df{i}"]    
    if len(df.columns) >= 157:
        columnas_seleccionadas = list(df.columns[:13]) + list(df.columns[145:157]) + [df.columns[-1]]
        globals()[f"df_accion_{i}"] = df.select(columnas_seleccionadas)
    else:
        print(f"DataFrame df{i} no tiene suficientes columnas para la operación.")

for i in range(1, len(archivos) + 1):
    df_accion = globals()[f"df_accion_{i}"]
    
    index_vars = list(df_accion.columns[:13]) + [df_accion.columns[-1]]
    value_vars = [col for col in df_accion.columns if col not in index_vars]
    
    melted_df = df_accion.unpivot(index=index_vars, on=value_vars).rename({
        "variable": "Variable",
        "value": "Valor"
    })
    
    melted_df = melted_df.with_columns(pl.lit("FC Acción").alias("FC Type"))
    
    globals()[f"melted_df_accion_{i}"] = melted_df
    
    print(f"Melt realizado para df_accion_{i} con columna 'FC Type':")
    print(melted_df)
    print("-" * 50)


Melt realizado para df_accion_1 con columna 'FC Type':
shape: (461_124, 17)
┌────────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────┬───────────┐
│ ID         ┆ Ult.       ┆ Agrupación ┆ Sub Agrup ┆ … ┆ Fecha_FC  ┆ Variable  ┆ Valor ┆ FC Type   │
│ ---        ┆ Eslabón    ┆ ---        ┆ ación     ┆   ┆ ---       ┆ ---       ┆ ---   ┆ ---       │
│ str        ┆ ---        ┆ str        ┆ ---       ┆   ┆ str       ┆ str       ┆ str   ┆ str       │
│            ┆ str        ┆            ┆ str       ┆   ┆           ┆           ┆       ┆           │
╞════════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════╪═══════════╡
│ 1000144CL  ┆ 1000144    ┆ ACCESORIOS ┆ LIMPIEZA  ┆ … ┆ 01.2024   ┆ 2024-02-0 ┆ 0.0   ┆ FC Acción │
│ MAYORISTA  ┆            ┆            ┆ Y CUIDADO ┆   ┆ S&OP      ┆ 1 00:00:0 ┆       ┆           │
│            ┆            ┆            ┆           ┆   ┆ Demanda   ┆ 0.6       ┆       ┆           │
│            ┆ 

LEER PARA CONCATENAR FC ANASTASIA

In [11]:
for i, archivo in enumerate(archivos, start=1):
    df = globals()[f"df{i}"]    
    if len(df.columns) >= 69:
        columnas_seleccionadas = list(df.columns[:13]) + list(df.columns[56:68]) + [df.columns[-1]]
        globals()[f"df_anastasia_{i}"] = df.select(columnas_seleccionadas)
    else:
        print(f"DataFrame df{i} no tiene suficientes columnas para la operación.")

for i in range(1, len(archivos) + 1):
    df_anastasia = globals()[f"df_anastasia_{i}"]
    
    index_vars = list(df_anastasia.columns[:13]) + [df_anastasia.columns[-1]]
    value_vars = [col for col in df_anastasia.columns if col not in index_vars]
    
    melted_df = df_anastasia.unpivot(index=index_vars, on=value_vars).rename({
        "variable": "Variable",
        "value": "Valor"
    })
    
    melted_df = melted_df.with_columns(pl.lit("FC Anastasia").alias("FC Type"))
    
    globals()[f"melted_df_anastasia_{i}"] = melted_df
    
    print(f"Melt realizado para df_anastasia_{i} con columna 'FC Type':")
    print(melted_df)
    print("-" * 50)


Melt realizado para df_anastasia_1 con columna 'FC Type':
shape: (461_124, 17)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ ID        ┆ Ult.      ┆ Agrupació ┆ Sub Agrup ┆ … ┆ Fecha_FC  ┆ Variable  ┆ Valor     ┆ FC Type  │
│ ---       ┆ Eslabón   ┆ n         ┆ ación     ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ str       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ str       ┆ str       ┆ str       ┆ str      │
│           ┆ str       ┆ str       ┆ str       ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 1000144CL ┆ 1000144   ┆ ACCESORIO ┆ LIMPIEZA  ┆ … ┆ 01.2024   ┆ 2024-02-0 ┆ 0.0       ┆ FC Anast │
│ MAYORISTA ┆           ┆ S         ┆ Y CUIDADO ┆   ┆ S&OP      ┆ 1         ┆           ┆ asia     │
│           ┆           ┆           ┆           ┆   ┆ Demanda   ┆ 00:00:00  ┆           ┆          │
│           

In [12]:
melted_dfs = [globals()[var] for var in globals() if var.startswith("melted_df_")]
df_concatenado = pl.concat(melted_dfs)
print("DataFrames concatenados:")
print([var for var in globals() if var.startswith("melted_df_")])
print("DataFrame concatenado:")
print(df_concatenado)


DataFrames concatenados:
['melted_df_fi_1', 'melted_df_fi_2', 'melted_df_fi_3', 'melted_df_fi_4', 'melted_df_fi_5', 'melted_df_fi_6', 'melted_df_fi_7', 'melted_df_fi_8', 'melted_df_fi_9', 'melted_df_fi_10', 'melted_df_fi_11', 'melted_df_fi_12', 'melted_df_fi_13', 'melted_df_adj_1', 'melted_df_adj_2', 'melted_df_adj_3', 'melted_df_adj_4', 'melted_df_adj_5', 'melted_df_adj_6', 'melted_df_adj_7', 'melted_df_adj_8', 'melted_df_adj_9', 'melted_df_adj_10', 'melted_df_adj_11', 'melted_df_adj_12', 'melted_df_adj_13', 'melted_df_accion_1', 'melted_df_accion_2', 'melted_df_accion_3', 'melted_df_accion_4', 'melted_df_accion_5', 'melted_df_accion_6', 'melted_df_accion_7', 'melted_df_accion_8', 'melted_df_accion_9', 'melted_df_accion_10', 'melted_df_accion_11', 'melted_df_accion_12', 'melted_df_accion_13', 'melted_df_anastasia_1', 'melted_df_anastasia_2', 'melted_df_anastasia_3', 'melted_df_anastasia_4', 'melted_df_anastasia_5', 'melted_df_anastasia_6', 'melted_df_anastasia_7', 'melted_df_anastasia

In [13]:
df_concatenado = df_concatenado.with_columns(
    df_concatenado["Variable"].str.slice(0, 10).alias("Variable")
)


In [14]:
import os
import polars as pl

ruta_carpeta = r"C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentoss\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Historico Inbound\Parquet_Inbound_Histórico"
archivos = [f for f in os.listdir(ruta_carpeta) if f.endswith('.parquet')]

def extraer_fecha(nombre):
    try:
        mes, año = map(int, nombre[:7].split('.'))
        return año, mes
    except ValueError:
        return 0, 0

# Detectar el archivo más reciente
archivo_ultimo = max(archivos, key=extraer_fecha)

print(f"Archivo leído: {archivo_ultimo}")

# Leer el archivo
SO = pl.read_parquet(os.path.join(ruta_carpeta, archivo_ultimo))

# Definir las columnas específicas a considerar como las "primeras"
primeras_columnas = [
    'ID', 'Ult. Eslabón', 'Agrupación', 'Sub Agrupación', 'Categoría', 
    'Sub Categoría', 'Descripcion Material', 'Marca', 'Sector', 'Vig', 
    'Nuevo', 'Season', 'Valor Libro'
]

# Verificar que las columnas existen en el DataFrame
primeras_columnas = [col for col in primeras_columnas if col in SO.columns]

# Identificar el índice de la columna "Prom 3-6-12" (en minúsculas)
columnas_lower = [col.lower() for col in SO.columns]
indice_prom = columnas_lower.index("prom 3-6-12")

# Mantener las 24 columnas a la izquierda de "Prom 3-6-12"
inicio_24 = max(indice_prom - 24, 0)
fin_24 = indice_prom
columnas_24_izquierda = SO.columns[inicio_24:fin_24]

# De esas 24 columnas, eliminar las últimas 12 y mantener las primeras 12
columnas_finales = columnas_24_izquierda[:12]

# Seleccionar las columnas finales junto con las primeras columnas especificadas y la última columna
columnas_seleccionadas = primeras_columnas + list(columnas_finales) + [SO.columns[-1]]
SO = SO.select(columnas_seleccionadas)

# Realizar un unpivot
index_vars = primeras_columnas + [SO.columns[-1]]
value_vars = [col for col in SO.columns if col not in index_vars]

melted_SO = SO.unpivot(index=index_vars, on=value_vars).rename({
    "variable": "Variable",
    "value": "Valor"
})

# Agregar columna "SELLOUT" con valor "SELLOUT"
melted_SO = melted_SO.with_columns(pl.lit("SELLOUT").alias("FC Type"))
melted_SO = melted_SO.with_columns(
    melted_SO["Variable"].str.slice(0, 10).alias("Variable")
)


# Mostrar el resultado
print("Resultado del unpivot para el DataFrame 'SO':")
print(melted_SO)


Archivo leído: 12.2024_S&OP_Demanda_Sin_restricciones_Inbound_Ciclo_Enero_2.parquet
Resultado del unpivot para el DataFrame 'SO':
shape: (594_372, 17)
┌────────────┬────────────┬────────────┬────────────┬───┬────────────┬───────────┬───────┬─────────┐
│ ID         ┆ Ult.       ┆ Agrupación ┆ Sub        ┆ … ┆ Fecha_FC   ┆ Variable  ┆ Valor ┆ FC Type │
│ ---        ┆ Eslabón    ┆ ---        ┆ Agrupación ┆   ┆ ---        ┆ ---       ┆ ---   ┆ ---     │
│ str        ┆ ---        ┆ str        ┆ ---        ┆   ┆ str        ┆ str       ┆ str   ┆ str     │
│            ┆ str        ┆            ┆ str        ┆   ┆            ┆           ┆       ┆         │
╞════════════╪════════════╪════════════╪════════════╪═══╪════════════╪═══════════╪═══════╪═════════╡
│ 1063609CL  ┆ 1063609    ┆ NEUMATICOS ┆ NEUMATICOS ┆ … ┆ 12.2024    ┆ 2023-12-0 ┆ 4     ┆ SELLOUT │
│ AUTOPLANET ┆            ┆            ┆ AUTOMOVIL  ┆   ┆ S&OP       ┆ 1         ┆       ┆         │
│            ┆            ┆            ┆ 

In [15]:
print(melted_SO.columns)

['ID', 'Ult. Eslabón', 'Agrupación', 'Sub Agrupación', 'Categoría', 'Sub Categoría', 'Descripcion Material', 'Marca', 'Sector', 'Vig', 'Nuevo', 'Season', 'Valor Libro', 'Fecha_FC', 'Variable', 'Valor', 'FC Type']


In [16]:
# Concatenar el nuevo DataFrame al existente
df_concatenado = pl.concat([df_concatenado, melted_SO], how="vertical")

# Mostrar el DataFrame concatenado
print("DataFrame concatenado actualizado:")
print(df_concatenado)


DataFrame concatenado actualizado:
shape: (24_994_116, 17)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ ID        ┆ Ult.      ┆ Agrupació ┆ Sub Agrup ┆ … ┆ Fecha_FC  ┆ Variable  ┆ Valor     ┆ FC Type  │
│ ---       ┆ Eslabón   ┆ n         ┆ ación     ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ str       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ str       ┆ str       ┆ str       ┆ str      │
│           ┆ str       ┆ str       ┆ str       ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 1000144CL ┆ 1000144   ┆ ACCESORIO ┆ LIMPIEZA  ┆ … ┆ 01.2024   ┆ 2024-02-0 ┆ 0.0       ┆ FC Final │
│ MAYORISTA ┆           ┆ S         ┆ Y CUIDADO ┆   ┆ S&OP      ┆ 1         ┆           ┆          │
│           ┆           ┆           ┆           ┆   ┆ Demanda   ┆           ┆           ┆          │
│           ┆           ┆       

In [17]:
from datetime import datetime
import polars as pl
fecha_actual = datetime.now().strftime("%Y%m%d")
ruta_archivo = f"C:\\Users\\etorres.DERCOPARTS\\DERCO CHILE REPUESTOS SpA\\Planificación y abastecimiento - Documentoss\\Planificación y Compras AFM\\S&OP Demanda\\Codigos Demanda\\Historico Inbound\\Inbound_Consolidado\\FC_Historico_{fecha_actual}.parquet"

# Guardar el DataFrame en formato Parquet
df_concatenado.write_parquet(ruta_archivo)

print(f"Archivo guardado en: {ruta_archivo}")

Archivo guardado en: C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentoss\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Historico Inbound\Inbound_Consolidado\FC_Historico_20241218.parquet


In [18]:
from datetime import datetime
import polars as pl
fecha_actual = datetime.now().strftime("%Y%m%d")
ruta_archivo = f"C:\\Users\\etorres.DERCOPARTS\\DERCO CHILE REPUESTOS SpA\\Planificación y abastecimiento - Documentoss\\Planificación y Compras AFM\\S&OP Demanda\\Codigos Demanda\\Historico Inbound\\Inbound_Consolidado\\melted_so.parquet"

# Guardar el DataFrame en formato Parquet
melted_SO.write_parquet(ruta_archivo)

print(f"Archivo guardado en: {ruta_archivo}")

Archivo guardado en: C:\Users\etorres.DERCOPARTS\DERCO CHILE REPUESTOS SpA\Planificación y abastecimiento - Documentoss\Planificación y Compras AFM\S&OP Demanda\Codigos Demanda\Historico Inbound\Inbound_Consolidado\melted_so.parquet
